In [ ]:
pip install pygbif

In [ ]:
# Paginar con pygbif (o requests). Ejemplo con pygbif:
# pip install pygbif

from pygbif import occurrences
import pandas as pd
import time
import math

# Parámetros de búsqueda (ajusta según tu consulta)
taxon = "Dasypus novemcinctus"    # ejemplo
limit = 300                       # límite máximo por petición
max_records = 3000                # cuántos registros quieres en total (ajusta)
sleep_seconds = 1.0               # espera entre peticiones (sé amable)

# 1) obtener taxonKey (opcional, si filtras por especie)
from pygbif import species
sp = species.name_backbone(name=taxon)
taxonKey = sp.get("usageKey")
print("taxonKey:", taxonKey)

# 2) calcular el número de iteraciones
total_to_fetch = max_records
n_pages = math.ceil(total_to_fetch / limit)

# 3) paginar y acumular resultados
rows = []
for i in range(n_pages):
    start = i * limit
    print(f"Request {i+1}/{n_pages} — offset={start}")
    try:
        res = occurrences.search(taxonKey=taxonKey, limit=limit, offset=start)
        results = res.get("results", [])
        if not results:
            print("No more results.")
            break
        rows.extend(results)
        # guarda parcial en disco por si interrumpes
        pd.json_normalize(rows).to_csv("gbif_paginado_partial.csv", index=False)
    except Exception as e:
        print("Error en petición:", e)
        # espera y reintenta
        time.sleep(5)
    time.sleep(sleep_seconds)

# 4) convertir a DataFrame final
df = pd.json_normalize(rows)
print("Registros descargados:", len(df))
df.to_csv("gbif_paginado_final.csv", index=False)


| Parámetro                 | Descripción                                  | Ejemplo                                                |
| ------------------------- | -------------------------------------------- | ------------------------------------------------------ |
| `scientificName`          | Nombre científico completo                   | `"Dasypus novemcinctus"`                               |
| `taxonKey`                | Clave taxonómica GBIF (preferible al nombre) | `2435099`                                              |
| `country` o `countryCode` | País (ISO-2)                                 | `"MX"`, `"BR"`, `"US"`                                 |
| `basisOfRecord`           | Tipo de registro                             | `"HUMAN_OBSERVATION"`, `"PRESERVED_SPECIMEN"`          |
| `year`                    | Año o rango de años                          | `"2000,2024"`                                          |
| `month`                   | Mes o rango                                  | `"5,9"`                                                |
| `hasCoordinate`           | Solo registros con coordenadas               | `True`                                                 |
| `hasGeospatialIssue`      | Excluir errores espaciales                   | `False`                                                |
| `decimalLatitude`         | Rango latitudinal                            | `"15,33"`                                              |
| `decimalLongitude`        | Rango longitudinal                           | `"-115,-90"`                                           |
| `geometry`                | Polígono o bounding box WKT                  | `"POLYGON((-105 20,-105 25,-100 25,-100 20,-105 20))"` |
| `depth`                   | Rango de profundidad marina                  | `"0,1000"`                                             |
| `elevation`               | Rango de elevación                           | `"0,3000"`                                             |
| `datasetKey`              | Filtrar por dataset específico (UUID GBIF)   | `"c6dd232e-5bdf-4f8f-91b0-9f715b6e5d00"`               |
| `eventDate`               | Fecha completa (ISO o rango)                 | `"2010-01-01,2020-12-31"`                              |
| `continent`               | Continente                                   | `"NORTH_AMERICA"`                                      |
| `projectId`               | ID de proyecto de observación                | `"BIOMEXICO_2020"`                                     |
| `occurrenceStatus`        | `"PRESENT"` o `"ABSENT"`                     | `"PRESENT"`                                            |


In [ ]:
# 📦 Instalación (solo una vez)
# !pip install pygbif

from pygbif import occurrences, species
import pandas as pd
import time
import math

# ============================
# ⚙️ CONFIGURACIÓN DE LA BÚSQUEDA
# ============================

# Especie o grupo de interés
taxon_name = "Dasypus novemcinctus"   # Armadillo
sp = species.name_backbone(name=taxon_name)
taxonKey = sp.get("usageKey")
print(f"🔍 TaxonKey de {taxon_name}: {taxonKey}")

# Parámetros principales
params = {
    "taxonKey": taxonKey,          # ID único de especie
    "country": "MX",               # Código ISO del país (ej: 'MX' para México)
    "hasCoordinate": True,         # Solo registros con coordenadas válidas
    "hasGeospatialIssue": False,   # Excluir registros con errores de georreferenciación
    "year": "2010,2025",           # Rango de años
    "basisOfRecord": "HUMAN_OBSERVATION",  # Tipo de registro
    "limit": 300,                  # Máximo permitido por solicitud
}

# 🗺️ Opcional: filtros espaciales
# Bounding box = área geográfica [minLon, minLat, maxLon, maxLat]
# Ejemplo: norte de México
params["decimalLatitude"] = "23,32"
params["decimalLongitude"] = "-110,-100"

# ============================
# 🔁 PAGINACIÓN AUTOMÁTICA
# ============================

max_records = 3000      # Cuántos registros totales deseas
sleep_seconds = 1.5     # Pausa entre peticiones (respetar API)
rows = []

n_pages = math.ceil(max_records / params["limit"])
print(f"📦 Descargando hasta {max_records} registros (~{n_pages} páginas)")

for i in range(n_pages):
    offset = i * params["limit"]
    print(f"➡️  Página {i+1}/{n_pages} — offset={offset}")
    try:
        res = occurrences.search(**params, offset=offset)
        results = res.get("results", [])
        if not results:
            print("⛔ No se encontraron más resultados.")
            break
        rows.extend(results)
    except Exception as e:
        print("⚠️ Error en la petición:", e)
        time.sleep(3)
    time.sleep(sleep_seconds)

# ============================
# 📊 CONVERTIR A DATAFRAME
# ============================

df = pd.json_normalize(rows)
print(f"✅ Registros descargados: {len(df)}")
print("📋 Columnas disponibles:")
print(df.columns.tolist()[:15])  # Muestra las primeras columnas

# Guardar CSV
df.to_csv("GBIF_ocurrencias_Dasypus.csv", index=False)
df.head()
